In [ ]:
# dive에 있는 폴더들을 코랩에 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 코랩에서 한글깨짐을 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (8,040 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121749 files and dire

### 상장기업 2년치 만들기

In [ ]:
path = '/content/drive/MyDrive/1차프로젝트/datasets/{}'
bring_df = pd.read_csv(path.format('상장승인+TS2000 기업  매칭/상장기업_1990_2023_찐막.csv'), encoding='cp949')

거래소코드 = bring_df['거래소코드'].unique()
len(거래소코드)

1796

In [ ]:
bring_df['회계년'] = bring_df['회계년도'].apply(lambda x : int(str(x)[:4]))

In [ ]:
len(bring_df['거래소코드'].unique())

1796

In [ ]:
concat_df = []
for code in 거래소코드:
  find_df = bring_df[bring_df['거래소코드'] == code]
  청구연도 = find_df['심사청구'].unique()[0]
  # print(청구연도)
  df = find_df[find_df['회계년'] < 청구연도]
  # print(df)
  concat_df.append(df)
final_df = pd.concat(concat_df)
final_df = final_df[final_df['회사명'] != '엔에이치엔(주)']

In [ ]:
print(len(final_df['거래소코드'].unique()))
print(len(bring_df['거래소코드'].unique()))


1696
1796


In [ ]:
final_df['재고자산 대 유동자산비율(IFRS)'] = (final_df['재고자산(요약)(IFRS)(백만원)'] / final_df['유동자산(*)(요약)(IFRS)(백만원)']) * 100
final_df['재고자산 대 유동자산비율(IFRS연결)'] = (final_df['재고자산(요약)(IFRS연결)(백만원)'] / final_df['유동자산(*)(요약)(IFRS연결)(백만원)']) * 100
final_df.drop(['재고자산(요약)(IFRS)(백만원)', '유동자산(*)(요약)(IFRS)(백만원)', '재고자산(요약)(IFRS연결)(백만원)', '유동자산(*)(요약)(IFRS연결)(백만원)', '기업규모코드'], axis=1, inplace=True)

In [ ]:
col_name = final_df.columns.tolist()

flag1 = col_name[45:91] # gaap
flag2 = col_name[91:136] # ifrs
flag3 = col_name[136:-4] # ifrs 연결

# flag1
flag2.append('재고자산 대 유동자산비율(IFRS)')
flag3.append('재고자산 대 유동자산비율(IFRS연결)')

In [ ]:
# final_df.isnull().sum().sort_values(ascending=False).head(40)

In [ ]:
ifrs = [
'자산(*)(요약)(IFRS)(백만원)',
'부채(*)(요약)(IFRS)(백만원)',
'자본(*)(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)',
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)',
'계속영업이익(손실)(요약)(IFRS)(백만원)',
'당기순이익(손실)(요약)(IFRS)(백만원)',
'투자활동으로 인한 현금흐름(*)(요약)(IFRS)(백만원)',
# '이익준비금(요약)(IFRS)',
# '거래소코드'
]

ifrs_link = [
'자산(*)(요약)(IFRS연결)(백만원)',
'부채(*)(요약)(IFRS연결)(백만원)',
'자본(*)(요약)(IFRS연결)(백만원)',
'매출액(수익)(요약)(IFRS연결)(백만원)',
'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)',
'계속영업이익(손실)(요약)(IFRS연결)(백만원)',
'당기순이익(손실)(요약)(IFRS연결)(백만원)',
'투자활동으로 인한 현금흐름(*)(요약)(IFRS연결)(백만원)',
# '이익준비금(요약)(IFRS연결)',
# '거래소코드'
]
gaap = [
'자산총계(요약)(백만원)',
'부채총계(요약)(백만원)',
'자본총계(요약)(백만원)',
'매출액(요약)(백만원)',
'법인세비용차감전(계속사업)손익(요약)(백만원)',
'영업이익(요약)(백만원)',
'당기순이익(요약)(백만원)',
'투자활동으로 인한 현금흐름(요약)(백만원)',
# '이익준비금(요약)(IFRS)'
# '거래소코드'
]
commom_fe = col_name[:12]

ifrs.extend(flag2)
ifrs_link.extend(flag3)
gaap.extend(flag1)



ifrs_link_to_remove = '사내유보율(IFRS연결)'
ifrs_to_remove = '사내유보율(IFRS)'
gaap_to_remove = '사내유보율'

ifrs = [x for x in ifrs if x != ifrs_to_remove]
ifrs_link = [x for x in ifrs_link if x != ifrs_link_to_remove]
gaap = [x for x in gaap if x != gaap_to_remove]

# ifrs_link_to_remove = '투자집중도(IFRS연결)'
# # ifrs_to_remove = '사내유보율(IFRS)'
# gaap_to_remove = '투자집중도'

# # ifrs = [x for x in ifrs if x != ifrs_to_remove]
# ifrs_link = [x for x in ifrs_link if x != ifrs_link_to_remove]
# gaap = [x for x in gaap if x != gaap_to_remove]

ifrs.extend(commom_fe)
ifrs_link.extend(commom_fe)
gaap.extend(commom_fe)

In [ ]:
gaap_df = final_df[gaap].dropna()
ifrs_df = final_df[ifrs].dropna()
ifrs_link_df = final_df[ifrs_link].dropna()

# 각 DataFrame에서 중복된 인덱스 확인
dup_index_df1_df2 = gaap_df[gaap_df.index.isin(ifrs_df.index)]
dup_index_df1_df3 = gaap_df[gaap_df.index.isin(ifrs_link_df.index)]
dup_index_df2_df3 = ifrs_df[ifrs_df.index.isin(ifrs_link_df.index)]

# 중복된 인덱스 출력
print("중복된 인덱스 (df1, df2):", len(dup_index_df1_df2.index.tolist()))
print("중복된 인덱스 (df1, df3):", len(dup_index_df1_df3.index.tolist()))
print("중복된 인덱스 (df2, df3):", len(dup_index_df2_df3.index.tolist()))

c1 = dup_index_df1_df2.index.tolist()
c2 = dup_index_df1_df3.index.tolist()
c3 = dup_index_df2_df3.index.tolist()

ifrs_df = ifrs_df[~ifrs_df.index.isin(c1)]
ifrs_link_df = ifrs_link_df[~ifrs_link_df.index.isin(c2)]
ifrs_link_df = ifrs_link_df[~ifrs_link_df.index.isin(c3)]

gaap_df['회계기준'] = 0
ifrs_df['회계기준'] = 1
ifrs_link_df['회계기준'] = 2

중복된 인덱스 (df1, df2): 1839
중복된 인덱스 (df1, df3): 743
중복된 인덱스 (df2, df3): 743


In [ ]:
gaap_rename_col = ['자산총계',
 '부채총계',
 '자본총계',
 '매출액',
 '법인세비용차감전(계속사업)손익',
 '영업이익',
 '당기순이익',
 '투자활동으로 인한 현금흐름',
 '총자본증가율',
 '영업이익증가율',
 '순이익증가율',
 '재고자산증가율',
 '자기자본증가율',
 '매출액증가율',
 '매출액영업이익률',
 '매출액순이익률',
 '총자본순이익률',
 '자기자본영업이익률',
 '자기자본순이익률',
 '기업순이익률',
 '금융비용부담률',
 'R & D 투자효율',
 '유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '비유동비율',
 '유동비율',
 '당좌비율',
 '부채비율',
 '차입금비율',
 '이자보상배율(이자비용)',
 '총자본회전률',
 '재고자산회전률',
 '매출채권회전률',
 '순운전자본회전률',
 '부가가치율',
 '유동자산증가율',
 '비유동자산증가율',
 '인건비 대 총비용비율',
 '재고자산 대 유동자산비율',
 '현금비율',
 '순운전자본비율',
 '차입금의존도',
  '투자집중도',
 'CASH FLOW 대 부채비율',
 'CASH FLOW 대 차입금비율',
 'CASH FLOW 대 총자본비율',
 'CASH FLOW 대 매출액비율',
 '자기자본회전률',
 '유동자산회전률',
 '재고자산회전기간',
 '비유동자산회전률',
 '총자본투자효율',
                   ]
ifrs_rename_col = ['자산총계',
 '부채총계',
 '자본총계',
 '매출액',
 '법인세비용차감전(계속사업)손익',
 '영업이익',
 '당기순이익',
 '투자활동으로 인한 현금흐름',
 '총자본증가율',
 '영업이익증가율',
 '순이익증가율',
 '재고자산증가율',
 '자기자본증가율',
 '매출액증가율',
 '매출액영업이익률',
 '매출액순이익률',
 '총자본순이익률',
 '자기자본영업이익률',
 '자기자본순이익률',
 '기업순이익률',
 '금융비용부담률',
 'R & D 투자효율',
 '유동자산구성비율',
 '자기자본구성비율',
 '타인자본구성비율',
 '비유동비율',
 '유동비율',
 '당좌비율',
 '부채비율',
 '차입금비율',
 '이자보상배율(이자비용)',
 '총자본회전률',
 '재고자산회전률',
 '매출채권회전률',
 '순운전자본회전률',
 '부가가치율',
 '유동자산증가율',
 '비유동자산증가율',
 '인건비 대 총비용비율',
 '현금비율',
 '순운전자본비율',
 '차입금의존도',
  '투자집중도',
 'CASH FLOW 대 부채비율',
 'CASH FLOW 대 차입금비율',
 'CASH FLOW 대 총자본비율',
 'CASH FLOW 대 매출액비율',
 '자기자본회전률',
 '유동자산회전률',
 '재고자산회전기간',
 '비유동자산회전률',
 '총자본투자효율',
'재고자산 대 유동자산비율',
]

gaap_rename_col.extend(commom_fe)
ifrs_rename_col.extend(commom_fe)

gaap_rename_col.append('회계기준')
ifrs_rename_col.append('회계기준')

In [ ]:
ifrs_df.columns = ifrs_rename_col
ifrs_link_df.columns = ifrs_rename_col
gaap_df.columns = gaap_rename_col

In [ ]:
dup_index_df1_df2 = gaap_df[gaap_df.index.isin(ifrs_df.index)]
dup_index_df1_df3 = gaap_df[gaap_df.index.isin(ifrs_link_df.index)]
dup_index_df2_df3 = ifrs_df[ifrs_df.index.isin(ifrs_link_df.index)]

# 중복된 인덱스 출력
print("중복된 인덱스 (df1, df2):", len(dup_index_df1_df2.index.tolist()))
print("중복된 인덱스 (df1, df3):", len(dup_index_df1_df3.index.tolist()))
print("중복된 인덱스 (df2, df3):", len(dup_index_df2_df3.index.tolist()))

중복된 인덱스 (df1, df2): 0
중복된 인덱스 (df1, df3): 0
중복된 인덱스 (df2, df3): 0


In [ ]:
ifrs_df.reset_index(drop=True)
ifrs_link_df.reset_index(drop=True)
gaap_df.reset_index(drop=True)


In [ ]:
df_1 =  pd.concat([ifrs_df, ifrs_link_df, gaap_df], axis=0)
el = len(df_1['거래소코드'].unique())
print(f'고유한 거래소코드 : {el}')

고유한 거래소코드 1285


In [ ]:
drop_cop_2 = list(set(final_df['거래소코드'].unique()) - set(df_1['거래소코드'].unique()))
drop_cop_2_df = final_df[final_df['거래소코드'].isin(drop_cop_2)]
drop = len(drop_cop_2_df['거래소코드'].unique())
print(f'소실된 기업의 수 : {drop}')

소실된 기업의 수 411


In [ ]:
# drop_cop_2_df.to_csv(path.format('상장승인+TS2000 기업  매칭/drop_411.csv'))

In [ ]:
year1_df = []
for i in 거래소코드:
  find_df = df_1[df_1['거래소코드'] == i].reset_index(drop=True)
  find_df.sort_values('회계년도', inplace=True)
  # print(find_df)
  if len(find_df) >= 1:
    find_df2 = find_df.iloc[-1, :]
    find_merge = pd.DataFrame(find_df2).T
    year1_df.append(find_merge)
  else:
    pass

In [ ]:
df_2 = pd.concat(year1_df, axis=0).reset_index(drop=True)
el = len(df_2['거래소코드'].unique())
print(f'1년치 재무가 있는 기업의 수 : {el}')

1년치 재무가 있는 기업의 수 : 1285


In [ ]:
year2_df = []
for i in 거래소코드:
  find_df = df_1[df_1['거래소코드'] == i].reset_index(drop=True)
  find_df.sort_values('회계년도', inplace=True)
  # print(find_df)
  if len(find_df) >= 2:
    find_df2 = find_df.iloc[-2, :]
    find_df3 = find_df.iloc[-1, :]
    # print(find_df2)
    find_merge = pd.concat([find_df2, find_df3], axis=0)
    find_merge = pd.DataFrame(find_merge).T
    year2_df.append(find_merge)
  else:
    pass
df_3 = pd.concat(year2_df, axis=0).reset_index(drop=True)
print(f'2년치 재무가 있는 기업의 수 : {len(df_3)}')

2년치 재무가 있는 기업의 수 : 1130


In [ ]:
cols = df_1.columns.tolist()

first_col = [item + '_1' for item in cols]
second_col = cols
first_col.extend(second_col)
df_3.columns = first_col

In [ ]:

df_3['회계년도_diff'] = df_3['회계년도'].astype(str).str[:4].astype('int') - df_3['회계년도_1'].astype(str).str[:4].astype('int')

In [ ]:
dup = df_3[df_3['회계년도_diff'] == 0][['회계년도_1', '회계년도', '회사명', '거래소코드']].sort_values('회사명')
find_code = dup['거래소코드'].tolist()
print(len(find_code))

12


In [ ]:
dup2 = df_1[df_1['거래소코드'].isin(find_code)]

for i in dup2['거래소코드'].unique():
  find = df_1[df_1['거래소코드'] == i]
  # print(find.info())
  if len(find) >= 3 :
    first = find.iloc[-1][['회계년도']].apply(lambda x: int(str(x).split('.')[0]))
    th = find.iloc[-3][['회계년도']].apply(lambda x: int(str(x).split('.')[0]))
    if (first[0] - th[0]) == 0:
      print(find['거래소코드'])
    print(first,th)

회계년도    2005
Name: 4042, dtype: int64 회계년도    2004
Name: 4040, dtype: int64
회계년도    2004
Name: 5429, dtype: int64 회계년도    2003
Name: 5427, dtype: int64
회계년도    1999
Name: 9418, dtype: int64 회계년도    1998
Name: 9416, dtype: int64
회계년도    2009
Name: 9826, dtype: int64 회계년도    2008
Name: 9824, dtype: int64
회계년도    2011
Name: 10006, dtype: int64 회계년도    2010
Name: 10004, dtype: int64
회계년도    1999
Name: 12260, dtype: int64 회계년도    1998
Name: 12258, dtype: int64
회계년도    1999
Name: 19900, dtype: int64 회계년도    1998
Name: 19898, dtype: int64
회계년도    2001
Name: 21999, dtype: int64 회계년도    2000
Name: 21997, dtype: int64
회계년도    2007
Name: 23520, dtype: int64 회계년도    2006
Name: 23518, dtype: int64
회계년도    2000
Name: 26596, dtype: int64 회계년도    1998
Name: 26594, dtype: int64
회계년도    2013
Name: 27181, dtype: int64 회계년도    2012
Name: 27179, dtype: int64


In [ ]:
year3_df = []
for i in dup2['거래소코드'].unique():
  find_df = df_1[df_1['거래소코드'] == i].reset_index(drop=True)
  find_df.sort_values('회계년도', inplace=True)
  # print(find_df)
  if len(find_df) >= 3:
    find_df2 = find_df.iloc[-3, :]
    find_df3 = find_df.iloc[-1, :]
    # print(find_df2)
    find_merge = pd.concat([find_df2, find_df3], axis=0)
    find_merge = pd.DataFrame(find_merge).T
    year3_df.append(find_merge)
  else:
    pass

In [ ]:
dz_0 = pd.concat(year3_df)
dz_0.columns = first_col
dz_0['회계년도_diff'] = dz_0['회계년도'].astype(str).str[:4].astype('int') - dz_0['회계년도_1'].astype(str).str[:4].astype('int')
df_4 = pd.concat([df_3, dz_0], axis=0).reset_index(drop=True)

In [ ]:
search_df = df_3[(df_3['회계년도_diff'] == 2) | (df_3['회계년도_diff'] == 1)]
len(search_df['거래소코드'].unique())

1107

In [ ]:
search_df = df_4[(df_4['회계년도_diff'] == 2) | (df_4['회계년도_diff'] == 1)]
el = len(search_df['거래소코드'].unique())
print(f'최종적으로 생존한 기업 : {el}')

최종적으로 생존한 기업 : 1118


In [ ]:
search_df.drop(['회계년도_diff'], axis=1, inplace=True)
search_df.columns
search_df.isnull().sum().sort_values(ascending=False)

자산총계_1           0
자기자본영업이익률        0
총자본회전률           0
이자보상배율(이자비용)     0
차입금비율            0
                ..
인건비 대 총비용비율_1    0
비유동자산증가율_1       0
유동자산증가율_1        0
부가가치율_1          0
회계기준             0
Length: 132, dtype: int64

In [ ]:
drop_cop_3_df = list(set(final_df['거래소코드'].unique()) - set(search_df['거래소코드'].unique()))
drop_cop_3_df = final_df[final_df['거래소코드'].isin(drop_cop_3_df)]
el = len(drop_cop_3_df['거래소코드'].unique())
print(f'Drop된 기업의 수 {el}')
# drop_cop_3_df.to_csv(path.format('상장승인+TS2000 기업  매칭/drop_578_new.csv'))

Drop된 기업의 수 578


In [ ]:
# drop_cop_3_df.to_csv(path.format('상장승인+TS2000 기업  매칭/drop_578.csv'))

In [ ]:
# search_df.to_csv(path.format('data_sets_final/상장기업(gaap)).csv'))
# search_df.to_csv(path.format('data_sets_final/상장기업2_a.csv'))
# df_1.to_csv(path.format('data_sets_final/상장기업1_a.csv'))

In [ ]:
# print(len(df_2.columns))
# print(len(df_3.columns))